## 라이브러리 임포트

In [1]:
from PIL import Image
import json
import os

## 이미지 및 JSON 파일 불러오기

In [2]:
def load_image_and_annotation(image_path, json_path):
    # 이미지 불러오기
    img = Image.open(image_path)
    
    # JSON 파일 불러오기 및 파싱
    with open(json_path, 'r', encoding='utf-8') as file:
        annotation = json.load(file)
    
    return img, annotation

## 어노테이션 정보 추출 및 데이터 준비

In [3]:
def prepare_data_from_annotations(img, annotation):
    annotations = annotation['Image_Annotation']['Annotations']
    for obj in annotations:
        # 다각형 어노테이션 정보에서 X, Y 좌표 추출
        x_coordinates = obj['X Coordinate']
        y_coordinates = obj['Y Coordinate']
        
        # 다각형의 바운딩 박스 계산
        x_min, y_min = min(x_coordinates), min(y_coordinates)
        x_max, y_max = max(x_coordinates), max(y_coordinates)
        
        # 이미지에서 해당 영역 잘라내기
        cropped_img = img.crop((x_min, y_min, x_max, y_max))
        
        # 잘라낸 이미지와 객체 이름(해충 종류) 반환
        object_name = obj['Object Name']
        yield cropped_img, object_name

## 전체 파일 처리하기

In [ ]:
# 예제 사용 방법
image_dir = '/Users/PC027/Desktop/이미지 분류 학습데이터/199.디지털 트랩 포집 해충 데이터/01-1.정식개방데이터/Training/01.원천데이터/TS_끈끈이형_미국선녀벌레'
json_dir = '/Users/PC027/Desktop/이미지 분류 학습데이터/199.디지털 트랩 포집 해충 데이터/01-1.정식개방데이터/Training/02.라벨링데이터/TL_끈끈이형_미국선녀벌레'
img_file = '20220808_113039_전북군산_미국선녀벌레(718).png'
json_file = '20220808_113039_전북군산_미국선녀벌레(718).json'

img_path = os.path.join(image_dir, img_file)
json_path = os.path.join(json_dir, json_file)

img, annotation = load_image_and_annotation(img_path, json_path)

# 어노테이션 정보를 기반으로 데이터 준비
for cropped_img, object_name in prepare_data_from_annotations(img, annotation):
    # 잘라낸 이미지(cropped_img)와 객체 이름(object_name)을 사용
    # 예를 들어, 이미지를 저장하거나 객체 이름을 출력할 수 있습니다.
    print(object_name)
    cropped_img.show()  # 이미지를 보고 싶다면 이 줄의 주석을 해제하세요.


## 어노테이션된 정보를 통해서 이미지 추출한 후 폴더에 저장하는 코드

In [4]:
import os
from PIL import Image
import json

# 각 폴더의 경로 설정
base_image_dir = '../segmentation/Splitted/train/images'  # image dir
base_json_dir = '../segmentation/Splitted/train/labels'  # json dir
output_base_dir = './image_dataset'  # save images dir

# 이미지 불러오기 및 어노테이션 파싱 함수
def load_image_and_annotation(image_path, json_path):
    img = Image.open(image_path)
    with open(json_path, 'r', encoding='utf-8') as file:
        annotation = json.load(file)
    return img, annotation

# 어노테이션 정보를 사용하여 데이터 준비 함수
def prepare_data_from_annotations(img, annotation, prefix=''):
    annotations = annotation['Image_Annotation']['Annotations']
    for count, obj in enumerate(annotations, start=1):
        try:
            x_coordinates = obj['X Coordinate']
            y_coordinates = obj['Y Coordinate']
            x_min, y_min = min(x_coordinates), min(y_coordinates)
            x_max, y_max = max(x_coordinates), max(y_coordinates)
            cropped_img = img.crop((x_min, y_min, x_max, y_max))  
            object_name = f"{prefix}_{count}"
        except:
            print(f"작업하지 못한 파일_{object_name}")
            pass
        else:
            yield cropped_img, object_name  # generator

# 주어진 폴더를 순회하며 모든 이미지와 JSON 파일에 대해 처리
for folder_name in os.listdir(base_image_dir):
    print(f"Processing {folder_name}...")
    image_dir = os.path.join(base_image_dir, folder_name)
    json_dir = os.path.join(base_json_dir, folder_name)
    output_dir = os.path.join(output_base_dir, f'추출이미지_{folder_name}')
    
    # 출력 디렉토리 생성
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # 폴더 내 모든 파일 처리
    for img_file in os.listdir(image_dir):
        if img_file.endswith('.png'):
            json_file = img_file.replace('.png', '.json')
            img_path = os.path.join(image_dir, img_file)
            json_path = os.path.join(json_dir, json_file)
            
            img, annotation = load_image_and_annotation(img_path, json_path)
            prefix = img_file.rsplit('.', 1)[0]
            
            for cropped_img, object_name in prepare_data_from_annotations(img, annotation, prefix):
                try:
                    output_path = os.path.join(output_dir, f"{object_name}.png")
                    cropped_img.save(output_path)
                except:
                    print(f"error has occurred at {object_name}")
                else:
                    print(f"Saved {object_name} to {output_path}")

Processing class1...
Saved 20220808_120039__135_1 to ./image_dataset\추출이미지_class1\20220808_120039__135_1.png
Saved 20220808_120039__135_2 to ./image_dataset\추출이미지_class1\20220808_120039__135_2.png
Saved 20220808_120039__631_1 to ./image_dataset\추출이미지_class1\20220808_120039__631_1.png
Saved 20220808_120039__631_2 to ./image_dataset\추출이미지_class1\20220808_120039__631_2.png
Saved 20220808_123039__139_1 to ./image_dataset\추출이미지_class1\20220808_123039__139_1.png
Saved 20220808_123039__740_1 to ./image_dataset\추출이미지_class1\20220808_123039__740_1.png
Saved 20220808_123039__740_2 to ./image_dataset\추출이미지_class1\20220808_123039__740_2.png
Saved 20220808_130039__377_1 to ./image_dataset\추출이미지_class1\20220808_130039__377_1.png
Saved 20220808_130039__377_2 to ./image_dataset\추출이미지_class1\20220808_130039__377_2.png
Saved 20220808_133039__629_1 to ./image_dataset\추출이미지_class1\20220808_133039__629_1.png
Saved 20220808_133039__629_2 to ./image_dataset\추출이미지_class1\20220808_133039__629_2.png
Saved 20220

# 추출된 데이터 정리
 - 학습 데이터, 검증 데이터, 테스트 데이터로 정리하는 코드 

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# 이미지 경로
base_dir = '/Users/PC027/Desktop/이미지 분류 학습데이터/199.디지털 트랩 포집 해충 데이터/01-1.정식개방데이터/Training'

# 이미지 폴더 이름
folders = [
    '추출이미지_TS_끈끈이형_갈색날개매미충',
    '추출이미지_TS_끈끈이형_미국선녀벌레',
    '추출이미지_TS_페로몬형_톱다리개미허리노린재'
]

# 폴더별로 작업 수행
for folder in folders:
    image_path = os.path.join(base_dir, folder)
    images = os.listdir(image_path)
    
    # 데이터를 분할합니다. 여기서는 학습:검증:테스트 = 70:15:15로 분할합니다.
    train_images, test_images = train_test_split(images, test_size=0.3, random_state=42)
    validation_images, test_images = train_test_split(test_images, test_size=0.5, random_state=42)
    
    # 분할된 데이터에 대한 폴더를 만듭니다.
    train_dir = os.path.join(base_dir, f"{folder}(학습데이터)")
    validation_dir = os.path.join(base_dir, f"{folder}(평가데이터)")
    test_dir = os.path.join(base_dir, f"{folder}(테스트데이터)")
    
    # 폴더 생성
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(validation_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)
    
    # 이미지 파일을 새로운 폴더로 이동
    for img in train_images:
        shutil.move(os.path.join(image_path, img), train_dir)
        
    for img in validation_images:
        shutil.move(os.path.join(image_path, img), validation_dir)
        
    for img in test_images:
        shutil.move(os.path.join(image_path, img), test_dir)